# This is the beginning of the DataHacks ScienceGang Project

## Part1: General Setup and Imports

In [1]:
# general imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
#import folium
#from folium import plugins

# These lines set up graphing capabilities.
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
pd.set_option('display.max_rows', 15)

%matplotlib inline

In [2]:
# read .csv file into a DataFrame
loc = 'U.S._Chronic_Disease_Indicators__CDI_.csv'
Chronic_Diseases = pd.read_csv(loc)
Chronic_Diseases

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2015,2015,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_2,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,11,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,66,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2012,2012,PR,Puerto Rico,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,72,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2012,2012,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519713,2014,2014,DE,Delaware,BRFSS,Arthritis,Physical inactivity among adults aged >= 18 ye...,NaN,%,Age-adjusted Prevalence,...,10,ART,ART3_0,AGEADJPREV,RACE,OTH,NaN,NaN,NaN,NaN
519714,2015,2015,OR,Oregon,BRFSS,"Nutrition, Physical Activity, and Weight Status",Meeting aerobic physical activity guidelines f...,NaN,%,Crude Prevalence,...,41,NPAW,NPAW11_3,CRDPREV,GENDER,GENF,NaN,NaN,NaN,NaN
519715,2014,2014,AR,Arkansas,State Inpatient Data,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,NaN,"cases per 10,000",Crude Rate,...,5,COPD,COPD5_1,CRDRATE,GENDER,GENF,NaN,NaN,NaN,NaN
519716,2016,2016,CT,Connecticut,BRFSS,Cardiovascular Disease,Influenza vaccination among noninstitutionaliz...,NaN,%,Age-adjusted Prevalence,...,9,CVD,CVD9_2,AGEADJPREV,RACE,HIS,NaN,NaN,NaN,NaN


In [3]:
# browse data structure and contents
print(Chronic_Diseases.shape)
print(Chronic_Diseases.describe())

(519718, 34)
           YearStart        YearEnd  DataValueAlt  LowConfidenceLimit  \
count  519718.000000  519718.000000  3.503350e+05       311062.000000   
mean     2013.141885    2013.162754  8.917742e+02           46.759401   
std         1.777622       1.762672  1.833073e+04           77.492628   
min      2001.000000    2001.000000  0.000000e+00            0.200000   
25%      2012.000000    2012.000000  1.845500e+01           12.700000   
50%      2013.000000    2013.000000  4.100000e+01           30.200000   
75%      2015.000000    2015.000000  7.030000e+01           55.400000   
max      2016.000000    2016.000000  2.600878e+06         1330.660000   

       HighConfidenceLimit     LocationID  
count        311062.000000  519718.000000  
mean             58.991287      30.993144  
std              88.668862      17.723341  
min               0.420000       1.000000  
25%              18.900000      17.000000  
50%              43.800000      30.000000  
75%              70.4

In [4]:
Chronic_Diseases.isna().sum()

YearStart                         0
YearEnd                           0
LocationAbbr                      0
LocationDesc                      0
DataSource                        0
                              ...  
StratificationID1                 0
StratificationCategoryID2    440529
StratificationID2            440529
StratificationCategoryID3    440530
StratificationID3            440530
Length: 34, dtype: int64

## Explore Empty Columns

In [5]:
responses = Chronic_Diseases['Response'].value_counts()
print(Chronic_Diseases.get("Response").isna().sum())
responses

440530


     79188
Name: Response, dtype: int64

In [6]:
responses = Chronic_Diseases['StratificationCategoryID2'].value_counts()
print(Chronic_Diseases.get("StratificationCategoryID2").isna().sum())
responses

440529


     79189
Name: StratificationCategoryID2, dtype: int64

In [7]:
responses = Chronic_Diseases['StratificationID2'].value_counts()
print(Chronic_Diseases.get("StratificationID2").isna().sum())
responses

440529


     79189
Name: StratificationID2, dtype: int64

In [8]:
responses = Chronic_Diseases['StratificationCategoryID3'].value_counts()
print(Chronic_Diseases.get("StratificationCategoryID3").isna().sum())
responses

440530


     79188
Name: StratificationCategoryID3, dtype: int64

In [9]:
responses = Chronic_Diseases['StratificationID3'].value_counts()
print(Chronic_Diseases.get("StratificationID3").isna().sum())
responses

440530


     79188
Name: StratificationID3, dtype: int64

In [10]:
responses = Chronic_Diseases['StratificationCategory3'].value_counts()
print(Chronic_Diseases.get("StratificationCategory3").isna().sum())
responses

440530


     79188
Name: StratificationCategory3, dtype: int64

In [11]:
responses = Chronic_Diseases['Stratification3'].value_counts()
print(Chronic_Diseases.get("Stratification3").isna().sum())
responses

440530


     79188
Name: Stratification3, dtype: int64

In [12]:
responses = Chronic_Diseases['StratificationCategory2'].value_counts()
print(Chronic_Diseases.get("StratificationCategory2").isna().sum())
responses

440530


     79188
Name: StratificationCategory2, dtype: int64

In [13]:
responses = Chronic_Diseases['Stratification2'].value_counts()
print(Chronic_Diseases.get("Stratification2").isna().sum())
responses

440530


     79188
Name: Stratification2, dtype: int64

In [14]:
responses = Chronic_Diseases['ResponseID'].value_counts()
print(Chronic_Diseases.get("ResponseID").isna().sum())
responses

440542


     79176
Name: ResponseID, dtype: int64

In [15]:
responses = Chronic_Diseases['DataValueFootnoteSymbol'].value_counts()
print(Chronic_Diseases.get('DataValueFootnoteSymbol').isna().sum())
responses

292400


****     98370
         56098
-        39252
~        30532
*         2062
         ...  
*, #         6
§,‡,^        1
§§           1
^^^          1
§,‡          1
Name: DataValueFootnoteSymbol, Length: 19, dtype: int64

Since there is no meaningful data in these columns, they will be dropped.

In [16]:
Chronic_Diseases = Chronic_Diseases.drop(columns = ['Response', 'StratificationCategoryID2', 'StratificationID2',
                        'StratificationCategoryID3', 'StratificationID3', 'StratificationCategory3',
                       'Stratification3', 'StratificationCategory2', 'Stratification2',
                       'ResponseID', 'DataValueFootnoteSymbol'])
Chronic_Diseases

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,...,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
0,2015,2015,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,NaN,59,ALC,ALC8_0_2,USD,OVERALL,OVR
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,0.09,...,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_1,USD,OVERALL,OVR
2,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,"(13.444304, 144.793731)",66,ALC,ALC8_0_1,USD,OVERALL,OVR
3,2012,2012,PR,Puerto Rico,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,"(18.220833, -66.590149)",72,ALC,ALC8_0_1,USD,OVERALL,OVR
4,2012,2012,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,NaN,59,ALC,ALC8_0_1,USD,OVERALL,OVR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519713,2014,2014,DE,Delaware,BRFSS,Arthritis,Physical inactivity among adults aged >= 18 ye...,%,Age-adjusted Prevalence,NaN,...,NaN,Race/Ethnicity,"Other, non-Hispanic","(39.008830667, -75.577741168)",10,ART,ART3_0,AGEADJPREV,RACE,OTH
519714,2015,2015,OR,Oregon,BRFSS,"Nutrition, Physical Activity, and Weight Status",Meeting aerobic physical activity guidelines f...,%,Crude Prevalence,39.5,...,42.00,Gender,Female,"(44.567449424, -120.155031326)",41,NPAW,NPAW11_3,CRDPREV,GENDER,GENF
519715,2014,2014,AR,Arkansas,State Inpatient Data,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,"cases per 10,000",Crude Rate,74.94,...,77.05,Gender,Female,"(34.748650124, -92.274490743)",5,COPD,COPD5_1,CRDRATE,GENDER,GENF
519716,2016,2016,CT,Connecticut,BRFSS,Cardiovascular Disease,Influenza vaccination among noninstitutionaliz...,%,Age-adjusted Prevalence,NaN,...,NaN,Race/Ethnicity,Hispanic,"(41.56266102, -72.649840952)",9,CVD,CVD9_2,AGEADJPREV,RACE,HIS


## Explore DataSources

In [17]:
Chronic_Diseases.get("DataSource").isna().sum()

0

### The dataset contains information from *32* different sources

In [18]:
# This dataset consists of 32 various sources
Chronic_Diseases.get("DataSource").value_counts().shape

(32,)

## Explore Stratification Categories

### Stratification Category 1 contains `RACE`,`GENDER`,`OVERALL`

In [19]:
Chronic_Diseases.get("StratificationCategoryID1").value_counts()

RACE       320170
GENDER     121660
OVERALL     77888
Name: StratificationCategoryID1, dtype: int64

### Stratification Category 2 contains all `NULL` and `BLANK` values

In [20]:
print(Chronic_Diseases.get("StratificationCategoryID2").isna().sum())
print(Chronic_Diseases.get("StratificationCategoryID2").value_counts())

AttributeError: 'NoneType' object has no attribute 'isna'

In [ ]:
# Drop the Stratification ID2 column
Chronic_Diseases.drop(columns="StratificationCategoryID2",inplace=True)

### Stratification Category 3 also contains all `NULL` and `BLANK` values

In [ ]:
print(Chronic_Diseases.get("StratificationCategoryID3").isna().sum())
Chronic_Diseases.get("StratificationCategoryID3").value_counts()

In [ ]:
# Drop the Stratification ID3 column
Chronic_Diseases.drop(columns="StratificationCategoryID3",inplace=True)

## Explore Stratification IDs

### Stratification ID1 contains 11 subcategories relating to `RACE`, `GENDER`, `OVERALL`

In [ ]:
print(Chronic_Diseases.get("StratificationID1").isna().sum())
print(Chronic_Diseases.get("StratificationID1").value_counts())

### Stratification ID2 contains all `NULL` and `BLANK`

In [ ]:
print(Chronic_Diseases.get("StratificationID2").isna().sum())
print(Chronic_Diseases.get("StratificationID2").value_counts())

In [ ]:
# Drop Stratification ID2 column
Chronic_Diseases.drop(columns="StratificationID2", inplace=True)

### Stratification ID3 contains all `NULL` and `BLANK`

In [ ]:
print(Chronic_Diseases.get("StratificationID3").isna().sum())
print(Chronic_Diseases.get("StratificationID3").value_counts())

In [ ]:
# Drop Stratification ID3 column
Chronic_Diseases.drop(columns="StratificationID3", inplace=True)

## Explore Stratification Numbers

In [ ]:
print(Chronic_Diseases.get("StratificationCategory1").isna().sum())
print(Chronic_Diseases.get("StratificationCategory1").value_counts())
print(Chronic_Diseases.get("StratificationCategory2").isna().sum())
print(Chronic_Diseases.get("StratificationCategory2").value_counts())
print(Chronic_Diseases.get("StratificationCategory3").isna().sum())
print(Chronic_Diseases.get("StratificationCategory3").value_counts())

In [ ]:
print(Chronic_Diseases.get("Stratification1").isna().sum())
print(Chronic_Diseases.get("Stratification1").value_counts())
print(Chronic_Diseases.get("Stratification2").isna().sum())
print(Chronic_Diseases.get("Stratification2").value_counts())
print(Chronic_Diseases.get("Stratification3").isna().sum())
print(Chronic_Diseases.get("Stratification3").value_counts())

In [ ]:
Chronic_Diseases.drop(columns=['Stratification2','Stratification3','StratificationCategory2','StratificationCategory3'],inplace=True)

## Next, let's see if there are redundant columns

In [ ]:
Chronic_Diseases.columns

In [ ]:
stratification_category = pd.DataFrame().assign(c1=Chronic_Diseases.get("StratificationCategory1"), c2=Chronic_Diseases.get("StratificationCategoryID1"))
stratification_category